Questions to address in this article
- Why do we need LSTM
- What is the intuition behind using LSTM
- How does it work
- Explain the vanishing and exploding gradients problem
- What are the advantages of using LSTM
- What are the disadvantages of using LSTM
- What is a better substitute
- What is the motivation behind using tanh
- Why do we need forget gate if we have already done selective write
- Does LSTM learn short term relations as well as long term relations
- 

With the help of this article I aim to achieve the following outcomes-
- 

The first thing that pops in our heads when we hear of sequential data is Recurrent Neural Networks or RNNs for short. A RNN is a modification of the traditional neural network with a specific goal in mind i.e. to perform well on learning tasks that involve sequential input and output data pairs such as speech recognition, language modelling, machine translation, time series forecasting or analysis, handwriting recognition, video analysis. Its architectural design allows it to scale easily to variable length inputs and outputs which is a major breakthrough. Each RNN hidden layer maintains a latent state that encodes the prior information it has seen so far. 
A word simply given as input carries no meaning but when woven into an article or even a sentence becomes meaningful and informative, it will most likely influence placement and use of other words as well. Just like we humans take prior as well as future information to make inference of the input and return some output, at given time step RNNs also utilize the prior (Vanilla RNN) or prior as well as future (Bidirectional RNNs) information to influence the input and output. This utility helps us train a model which is able to learn temporal orders and relations between different parts of the sequence. 

#### What makes them different from Multi Layered Perceptrons?

1. Multi Layered Perceptrons bear the limitation of dealing with <b>static input patterns</b> only i.e. taking in input data of a specific predefined dimensionality. This is because of the fact that each node has its own set of weights in a MLP. Whereas on the other hand RNNs are based on the idea of parameter sharing between different time steps which allows them to easily work and scale with variable length inputs without adding any extra parameters. Just like weight sharing in CNNs makes them shift equivarient i.e. the network can detect a particular feature anywhere in the image, the recurrent structure of RNNs allow them to leverage this concept to learn information irrespective of their position in the input sequence. For example consider the two statements -<br>
"Neal went to Kerala during the summer break of 2013."<br>
"In 2013, Neal went to kerala during his summer break."<br>
Now we want to extract the year from the given sequences. This can be done by using a RNN as given enough examples they will be able to learn the dependencies between different parts of the sequence.

2. The feedforward step in the traditional neural networks takes in all the input features at once for computation of the hidden layer's output whereas the RNNs are devised in such a way that each input variable is passed sequentially and at any given time step the present input variable and the output of the hidden or recurrent layer is influenced by the prior inputs whose learned information is encoded in the latent (or hidden) state vector. Adding another recurrent layer to the RNN network ,but this time traversing the input sequence in the negative direction allows the input variable at present time step to learn from the prior as well as future inputs. This modified learning is known as Bidirectional RNN. For example, in Speech recognition correct interpretation of a phoneme may depend on the next few phonemes due to co-articulation and may even depend on the next few words because of the linguistic dependencies between nearby words.

3. Time Complexity

#### Challenges faced by RNNs

RNNs are strong dynamic systems that can be used to map input sequences to output sequences and are very powerful in their ability to represent context but theoretical and emperical analysis suggests that it is very difficult to train them to learn long term dependencies between different parts of the input sequence using the infamous gradient descent technique. The reason being that gradients propagated over long distances or several time steps tend to either vanish or blow up irrespective of the input values.But gradient descent alone is not responsible for this challenge of learning long range depenedencies. 
<br>
We will now see that as the difference between time steps increase it becomes more and more difficult to propagate information even in the forward pass of the RNN.
<br>
Let us consider a linear RNN system (we will extrapolate for non-linear systems later) with only one hidden layer for a single sample input sequence. We already know that the hidden state vector $h^{t-1}$ at time step t-1 is used to compute one $h^{t}$ at time step t. <br>

$h^{(t)} = W_h^Th^{(t-1)} + W_x^Tx^t$ <br>
$h^{(t-1)} = W_h^Th^{(t-2)} + W_x^Tx^{(t-1)}$ <br>
.<br>
.<br>
.<br>
$h^{(0)} = W_h^Th^{(-1)} + W_x^Tx^{(0)}$ <br>
Hence,<br>
$h^{(t)} = (W_h^T)^{t+1}h^{(-1)} + (W_h^T)^{t}W_x^Tx^{(0)} + (W_h^T)^{t-1}W_x^Tx^{(1)} + ... + W_x^Tx^{(t)}$ <br>

$x^{(t)}$ is the input vector of size $Mx1$, $h^{(t)}$ is the hidden vector of size $Mx1$  at time step t and $W_h$ & $W_x$ are the corresponding parameter matrices of size $NxN$ and $MxN$ respectively assuming the number of units in the hidden layer are $N$.

We can simply remove the first term in the above equation as the initial value of the latent vector is always intialized to 0. 
<br>
$h^{(t)} = (W_h^T)^{(t)}W_x^Tx^{(0)} + (W_h^T)^{(t-1)}W_x^Tx^{(1)} + ... + W_x^Tx^t$ <br>

Hmm...Okay now let's pause and take a moment to think what new inferences can we make from this equation.
<br>
If you are thinking that somehow the exponential values of the weight matrices will dominate the input values then you are at the right place my friend. If not, try thinking about the equation using scalar values instead of vector representation. <br>
<img src = "images/stability graph.png" height = 200 width = 400> <br>
For the purpose of illustration, I have imputed different values of $x^{(0)}$ only as it is a part of the term with highest power and also their will be no change in the concept I am trying to illustrate you can take my word for it or try yourself.<br>
It is evident from the graphs that the value of the hidden state vector solely depends on the weight matrix and the time step. The input values have no influence on it. 

Now we will look at the vector notation because afterall that is what we are after.Right?<br>

Here also we will analyze the term with input x at time step 0.

If $W_h$ is a diagonalizable matrix that can be decomposed into its eigenvectors and eigenvalues, then the vectorized term $h_t$ would be-<br>

$h^{(t)} = (Q^T\lambda^{t}Q)W_x^Tx^{(0)}$<br>

OR<br>

$h^{(t)} = (Q^T\lambda^{t}Q)h^{(0)}$ since $h^{(-1)}$ is zero vector<br>

where $Q$ is the square matrix whose columns are the eigenvectors of $W$, and $\lambda$ is the diagonal matrix whose diagonal elements are the corresponding eigenvalues.<br>

One simple observation we can make here is that the eigenvalues are raised to the power $t$, causing the eigenvalues with value less than 1 to decay and those with magnitude greater than 1 to explode. Apart from this the hidden vector $h^{(0)}$ is getting projected onto a new basis formed by the eigenvectors of the weight matrix from which we can draw that any component of $h^{(0)}$ that does not align the with the largest eigenvector will eventually die out. Hence, for any input and large $t$, the length of the hidden vector will depend on the $t^{th}$ power of the eigenvalues and not on the input.

Since we have now understood the implications of the parameters in linear systems we can informally extrapolate for non-linear systems as the formal analysis requires more advance techniques. We will look at three different non-linear activation functions widely used in deep learning - sigmoid, hyperbolic tangent and ReLU. 
Again we will consider scalar values for convenience and visualization.
The graphs for the $h^{(t)}$ computed using sigmoid activation function show that the hidden value depends only on the bias term and not on the inputs

<em>Incomplete</em>

This scalar analysis also reasons why tanh functions are preferable over sigmoid and ReLU. Although not significantly advantageous, tanh functions are able to retain the information in memory for longer durations in comparison to the other two activations. 

Now, let us a take a look at the notorious and aforementioned issue of vanishing and exploding gradients. For those who are alien to it, vanishing gradients is a situation when the values of gradients of the loss function that are responsible to propagate error from the output to the layer parameters gets infinitesimal as network depth increases whereas on the other hand, exploding gradients means that the gradient terms shoot up to infinity which also makes training the weights challenging and very very slow. But the latter problem is quite easy to resolve as we can set a maximum value for the gradients and clip them whenever they try to escape the threshold. We know that the said problem is not novel to Recurrent Neural Networks. Vanishing and exploding gradients hamper training of every other type of neural network. The ML community has come up with multiple solutions and has been able to solve this issue for different network types such as Residual Connections or activation functions such as ReLU in CNNs.

The thing is that the likelihood of RNNs suffering from the vanishing gradients problem is almost certain due to a multitude of reasons and also has other consequences than just slower training. Tanh functions that we saw being more favourable in recurrent networks have first order gradients in the range [0,1] and mostly less than 1. This means that multiplication by the Jacobian of the activation units in the hidden layer is a shrinking operation, and since the input sequences in RNNs are very large and the multiplication operation is proportional to the lenght of the sequence, the result will always be close to zero. But till now we did not take into account the common weight matrix across the entire depth of the hidden layer which will be a part of the gradient (as we have seen earlier while computing $h^{(t)}$. This combined multiplication of the weight matrix with the gradient of the activation function will have the final gradient value blow up along the component of the weight matrix that has eigenvalues greater than 1 and vanish for those less than 1. Based on this we can say that the network is unable to communicate errors over long distances, store information for longer durations and thus, is unable to learn long term dependencies.     

Theoretical and empirical evidence from Bengio et al conveys that their is a trade off between the model learning long term dependencies and it being efficiently trained by gradient descent in reasonable time. Since the gradient of a long-term interaction is exponentially smaller in magnitude than the one of a small-term interaction, small noise in the latter can affect the former as stored information needs to be protected for longer and longer durations. Also more and more already learned outputs need protectection from such perturbations. They strongly suggest that in order for the model to learn long term dependencies it should enter the parameter space where the gradients vanish. This means not that it is impossible for the network to learn, but that it might take a very very long time to learn long-term dependencies, because the signal about those dependencies will tend to be hidden by the smallest fluctuations arising from short-term dependencies. 

So what do we do?

We need a technique that allows to selectively read and write information to the memory so that the network retains only the information that is relevant to our output and robust to noise in the input sequences. Also we want the ability to forget information from the memory once it is no longer relevant to the our prediction. Yes, I know what you are thinking. Something like a human brain. Just like the way our brain is able to retain the relevant details of a sentence or a paragraph or any other event for that matter, we need a memory manipulation mechanism that is efficient in learning, retaining and forgetting information if needed for long input sequences. 

Here comes the Long Short Term Memory or LSTM for short

### LSTM 

We humans can watch the entire series of Game of Thrones or the Lord of the Rings Triology or read fictional book collections such as the Harry Potter and remember details that help us understand contexts and establish connection between two ends of the media. We can remember intricate details in suspense thrillers and ming numbing movies like Interstellar without any careful effort because of the power of our brain to perform tasks like learning, storing, retreiving and forgetting simultaneously and effortlessly but our fellow machines although successful at many, struggle with descent performance at these taks. LSTMs were one successful effort of researchers Hochreiter and Schmidhuber to bridge or reduce the gap between the two. 
LSTMs are specialized RNNs that use unusual but beautifully engineered recurrent cells also known as memory cells to excel at the task of learning long-term dependencies. Before we dive deep, LSTMs in a nutshell contain a path known as the Constant Error Corrousal devised to facilitate constant error flow over long distances as and when needed along with specialized gates that open and close to allow or restrict access to the constant error flow. Each memory cell can be thought of as a small neural network comprising of different units working together to accomplish a task. Let's now look into each component in order.

#### Constant Error Corrousal

I would like to try a simple analogy to couple with the explanation of CEC. Let us think of CEC as a giant conveyer belt at the baggage claim in an airport. As the belt starts, the airport staff is irregularly but constantly placing bags (information) onto it.  Now as the belt (CEC) moves the bags on the belt stay undisturbed until a passenger picks her/his baggage (related information). This is the underlying idea of CEC. It allows error or information to flow untouched over long distances by preventing unwanted and unnecessary changes to the gradient. It is a memory band or tape that enables:
- memorizing information that influences the output of the network at a later time step in the future,
- prevention of unnecessary changes to the memory,
- unloading of information that is no longer relevant,
- reading information learnt way back in the past or at the beginning of the sequence.

#### Gates
In order to learn long term dependencies and efficiently utilize the memory bandwidth available we ideally wish to,
- Ignore or forget some of the input words or older knowledge
- Selectively read from the already learnt information
- Selectively write information to the error corrousal from th current input 

Here come the second most important piece of the memory cell - The gates. These gates are nothing but sigmoid units whose values range between 0 and 1 (inclusive) tasked with regulating the flow of information to and from the CEC as well as the hidden state vectors.
There are 3 different gates in a memory cell:
1. Forget Gate
2. Input Gate
3. Output Gate

Instead of going through each gate in order of their occurance in the cell. I will begin by talking about the tasks we wish to accomplish and discuss the corresponding to that task. 

##### Selective Writing

In vanilla RNNs we mindless passed the learned hidden state vector to the cell at the next time step but in LSTMs we do not want to do that. We want to pass only the relevant information to the next time step. This is why we introduce a new gate known as the output gate that carefully filters the amount of information from the CEC to be passed to the next time step. It does not make any changes to the Error Corrousal.

#### Lets have a look at backpropagation in time algorithm for a Many-to-One Recurrent Neural Network

#### Notations we will use:

$h^{(t)}$ - Output $h$, of the hidden layer at time step $t$<br>

$x^{(t)}$ - Input $x$ at time step $t$<br>

$o^{(t)}$ - Output $o$ of the output layer at time step $t$<br>

$z^{(t)}$ - Output of the hidden layer before applying the activation function at time step $t$<br>

$q^{(t)}$ - Output of the output layer before applying the activation function at time step $t$<br>

$y^{(t)}$ - Target value at time step $t$<br>

$W_{hx}$ - Weight matrix to learn the hidden state vector from the inputs<br>

$W_{hh}$ - Weight matrix to learn the hidden state vector at time step $t$ from one at time step $t-1$<br>

$W_{ho}$ - Weight matrix to generate the output from the hidden state vector<br>

$L\{y,o\}$ - Is the binary cross entropy loss between the predicted and target values. <br>

$T$ - Last time step<br>

$t_0$ - Initial time step<br>

<em>Please note I shall not divulge in solving the gradients as it might become quiet overwhelming for the readers.<em>

#### Steps Involved-

1. Begin by computing the gradient $\nabla_{o^{(T)}}L$, of the loss function with respect to the output value.
2. Next we compute the gradient $\nabla_{q^{(T)}}L$,, of the loss function with respect to $q^{(T)}$ which can be easily done using the chain rule:<br>

    $\nabla_{o^{(T)}}L$ = $\frac{\delta L}{\delta o^{(T)}}$ $\frac{\delta o^{(T)}}{\delta q^{(T)}}$<br>

3. Compute the gradient $\nabla_{h^{(T)}}L$:<br>
    
    $\nabla_{h^{(T)}}L$ = $\frac{\delta L}{\delta q^{(T)}}$ $W_{ho}$<br>

Three basic requirements for a parametric dynamic system that can learn to store relevant state information-
- That the system be able to store information for an abitrary duration.
- That the system be resistant to noise (fluctutations of the inputs that are random or irrelevant to predicting a correct output).
- That the system parameters be trainable (in reasonable time).

The theoretical results show that either  the RNN system is stable and resistant to noise or it is efficiently trainable by gradient descent but not both.


Many ANN structures have been proposed in the literature
to deal with time varying patterns. Multilayer perceptrons
(MLP’s) have the limitation that they can only deal with
static data patterns (i.e., input patterns of a predefined dimensionality), which requires definition of the size of the input
window in advance. Waibel et al. [16] have pursued time delay
neural networks (TDNN’s), which have proven to be a useful
improvement over regular MLP’s in many applications. The
basic idea of a TDNN is to tie certain parameters in a regular
MLP structure without restricting the learning capability of the
ANN too much. Recurrent neural networks (RNN’s) [5], [8],
[12], [13], [15] provide another alternative for incorporating
temporal dynamics

<footer>
    <h3>References</h3>
    <ul>
        <li>Deep Learning Book by Ian Goodfellow et al.</li>
        <li>Fundamentals of Recurrent Neural Network (RNN) and Long Short-Term Memory (LSTM) Network, <em>Alex Sherstinsky</em></li>
        <li>Long Short-Term Memory, <em>Hochreiter et al.</em></li>
        <li>Bidirectional Recurrent Neural Networks, <em>Schuster et al.</em></li>
    </ul>
    </footer>